In [1]:
import pandas as pd
from src.isotope_correction import (
    calc_correction_factor,
    calc_probability_table,
    calc_isotopologue_correction,
    calc_transition_prob,
    get_metabolite_formula,
)

In [2]:
import src.isotope_correction as ic

In [3]:
def fwhm(m_cal, m_z, resolution):
    """Calculate Full width half maximum (FWHM)."""
    return m_z ** (3 / 2) / (resolution * m_cal ** 0.5)


def min_mass_diff(m, z, m_cal, resolution):
    """Calculate minimal resolvable mass difference.
    
    For m and z return minimal mass difference that ca be resolved properly.
    :param m: float
        Mass of molecule
    :param z: int
        Charge of molecule
    :param m_cal: float
        Mass for which resolition has been determined
    :param resolution: float
        Resolution
    :returns: float
    """
    m_z = m / z
    return 1.66 * z * fwhm(m_cal, m_z, resolution)


def assign_light_isotopes(metabolite_series):
    """Replace all element names with light isotopes ("C" -> "C13")."""
    result = metabolite_series.rename(
        {
            "H": "H01",
            "C": "C12",
            "N": "N14",
            "O": "O16",
            "Si": "Si28",
            "P": "P31",
            "S": "S32",
        },
    )
    return result


def subtract_label(metabolite_series, label_series):
    """Subtract label atoms from metabolite formula."""
    formula_difference = metabolite_series.copy()
    iso_dict = {
        "H02": "H01",
        "C13": "C12",
        "N15": "N14",
    }
    for heavy in label_series.keys():
        light = iso_dict[heavy]
        formula_difference[light] = metabolite_series[light] - label_series[heavy]
    if any(formula_difference < 0):
        raise ValueError("Too many labelled atoms")
    return formula_difference


def get_isotope_mass_series(isotopes_file):
    """Get series of isotope masses."""
    return pd.read_csv(
        isotopes_file,
        sep="\t",
        usecols=["mass", "isotope"],
        index_col="isotope",
        squeeze=True,
    )


def calc_isotopologue_mass(metabolite_name, label, isotope_mass_series):
    """Calculate mass of isotopologue.
    
    Given the metabolite name and label composition, return mass in atomic units.
    :param metabolite_name: str
        Name as in metabolite_file
    :param label: str
        "No label" or formula, can contain whitespaces
    :param isotope_mass_series: pandas Series
        Isotope name (e.g. 'H02') as index and mass as values
        Output of 'get_isotope_mass_series'
    :returns: float
    """
    label = pd.Series(ic.parse_label(label), dtype="int64")
    metab = pd.Series(
        ic.get_metabolite_formula(metabolite_name, "src/metabolites.csv"), dtype="int64"
    )
    metab = assign_light_isotopes(metab)
    light_isotopes = subtract_label(metab, label)
    formula_isotopes = pd.concat([light_isotopes, label])
    mass = isotope_mass_series.multiply(formula_isotopes).dropna().sum()
    return mass


def is_isotologue_overlapp(label1, label2, metabolite_name, min_mass_diff, isotope_mass_series):
    """Return True if label1 and label2 are too close to detection limit.
    
    Checks whether two isotopologues defined by label and metabolite name
    are below miniumum resolved mass difference.
    """
    mass1 = calc_isotopologue_mass(metabolite_name, label1, isotope_mass_series)
    mass2 = calc_isotopologue_mass(metabolite_name, label2, isotope_mass_series)
    return abs(mass1 - mass2) < min_mass_diff

In [4]:
labels = {
    "No label": "No label",
    "2C13": "2C13",
    "2C13 3H02": "5C13",
    "4C13": "4C13",
    "4C13 3H02": "7C13",
    "4C13 6H02": "10C13",
}

In [5]:
name_his = "K(ac)QLATK(ac)AAR"
histo = get_metabolite_formula(name_his, "src/metabolites.csv")
calc_transition_prob("4C13", "7C13", histo, "src/metabolites.csv", "src/isotopes.csv")

0.009244545730428034

In [6]:
import itertools
for la1, la2 in itertools.permutations(labels.keys(), 2):
    res = calc_transition_prob(labels[la1], labels[la2], histo, "src/metabolites.csv", "src/isotopes.csv")
    if res != 0:
        print(f"{la1} -> {la2}: \t{res}")

No label -> 2C13: 	0.07381426583499244
No label -> 2C13 3H02: 	0.00012368777651995536
No label -> 4C13: 	0.001361418750680817
No label -> 4C13 3H02: 	5.649797437808613e-07
No label -> 4C13 6H02: 	5.443993706160078e-11
2C13 -> 2C13 3H02: 	0.010438038089519907
2C13 -> 4C13: 	0.06893425288359174
2C13 -> 4C13 3H02: 	0.00010012540064801634
2C13 -> 4C13 6H02: 	2.0673891041025335e-08
2C13 3H02 -> 4C13 3H02: 	0.06171258832061438
2C13 3H02 -> 4C13 6H02: 	7.135751950892934e-05
4C13 -> 2C13 3H02: 	0.28912220025087565
4C13 -> 4C13 3H02: 	0.009244545730428034
4C13 -> 4C13 6H02: 	5.344678211218808e-06
4C13 3H02 -> 4C13 6H02: 	0.007600783060074407


In [8]:
isotope_mass_series = get_isotope_mass_series("src/isotopes.csv")
min_mass = min_mass_diff(1070, 2, 200, 6E4)
is_isotologue_overlapp("7C13", "4C13 3H02", name_his, min_mass, isotope_mass_series)

True

In [9]:
mass_his = calc_isotopologue_mass(name_his, "No label", isotope_mass_series)
min_mass_diff(mass_his, 2, 200, 6E4)

0.04846043453279501

In [10]:
for i in range(22):
    if not is_isotologue_overlapp(str(i)+"C13", str(i)+"H02", name_his, min_mass, isotope_mass_series):
        print(f"Resolution good enough to separate H from C with {i} labeled atoms")
        break

Resolution good enough to separate H from C with 17 labeled atoms
